-- https://medium.com/@sevketay09/gemini-pro-ile-multimodal-rag-langchain-chroma-868a379c85b8<br>
-- https://www.linkedin.com/pulse/extract-text-from-any-pdf-image-dataset-generation-jagrat-patel/<br>
-- https://medium.com/@princekrampah/prompt-templates-in-langchain-248c015be3e0<br>
-- https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed<br>
-- https://medium.com/@epappas/dealing-with-vector-dimension-mismatch-my-experience-with-openai-mbeddings-and-qdrant-vector-20a6e13b6d9f

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
import google.generativeai as genai
from IPython.display import display, Markdown, clear_output
import textwrap
import os
import warnings
import time
from langchain_community.llms import HuggingFaceHub

In [2]:
warnings.filterwarnings("ignore")

In [3]:
from dotenv import load_dotenv
file_path = "./sample_01.pdf"
genai.configure(api_key = os.environ['GOOGLE_API_KEY'])

In [4]:
def to_markdown(text):
  text = text.replace('•','*')
  return Markdown(textwrap.indent(text, '>', predicate=lambda _: True))

In [5]:
def pdf2text(file_path):
    from langchain_community.document_loaders import PyPDFLoader
    loader = PyPDFLoader(file_path)
    return loader.load()

In [6]:
def pypdf2(file_path):
    from PyPDF2 import PdfReader
    pdf_reader = PdfReader(file_path)
    raw_text = ''
    for i, page in enumerate(pdf_reader.pages):
      text = page.extract_text()
      if text:
        raw_text += text
    return raw_text

In [7]:
def pdf_text2text(file_path):
    from langchain_community.document_loaders import UnstructuredFileLoader
    loader = UnstructuredFileLoader(file_path)
    documents = loader.load()
    # return '\n'.join(doc.page_content for doc in documents)
    return documents

In [8]:
def text_split(docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=4096, chunk_overlap=1000,separators=["\n\n", "\n", " ", ""])
    return text_splitter.split_documents(docs)

In [9]:
def vectorization(splitted_docs):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    return Chroma.from_documents(documents=splitted_docs, embedding=embeddings,persist_directory='db')

In [10]:
def default_model(prompt):
    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt)
    display(to_markdown(response.text))

In [17]:
default_model("Güneş nedir kısaca açıkla")

>Güneş, gezegenleri ve diğer nesneleri içeren Güneş Sistemimizin merkezindeki dev, sıcak ve parlak bir yıldızdır. Güneş, esas olarak hidrojen ve helyumdan oluşan devasa bir gaz topudur.
>
>* **Yakıt:** Güneş'in enerjisi, çekirdeğinde gerçekleşen nükleer füzyon reaksiyonlarından gelir. Bu reaksiyonlarda hidrojen helyuma dönüştürülür ve muazzam miktarda enerji açığa çıkar.
>* **Sıcaklık:** Güneş'in çekirdeğindeki sıcaklık yaklaşık 27 milyon Kelvin'dir ve yüzey sıcaklığı yaklaşık 5.500 Kelvin'dir.
>* **Büyüklük:** Güneş, Dünya'dan yaklaşık 109 kat daha büyük bir çapa sahiptir.
>* **Parlaklık:** Güneş, Dünya'dan gelen toplam güneş ışığının yaklaşık %99,9'unu yayar.
>* **Yerçekimi:** Güneş, Güneş Sistemindeki tüm nesneleri yörüngelerinde tutan güçlü bir yerçekimine sahiptir.
>* **Hayat için hayati önem:** Güneş, Dünya üzerinde yaşam için gerekli olan ışık, ısı ve enerjiyi sağlar.

In [12]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [13]:
context = pdf2text(file_path)
vector_index = vectorization(text_split(context))

In [14]:
prompt_template = """
  Soruyu verilen bağlama göre en anlaşılır ve detaylı şekilde cevapla.
  Gelen sorular karşılaştırma sorusu, genel sorular veya direk bilgi istenen sorular olabilir.
  Karşılaştırma sorularına bağlamdan anlamlı bir sonuç çıkararak cevap vereceksin.
  Soruları yanıtlarken sadece Türklerin bakış açısından cevapla.
  Sana sağlanan dokümanlarda bilgisi bulunmayan bir bağlama yanıt olarak "Metinde bilgi bulunmamaktadır" veya "Bilmiyorum" cevabını vereceksin.
  Markdown dosyası olarak çıktı ver.

  Context:\n {context}?\n
  Question: \n{question}\n

  Answer:
"""

# prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt_template)
model = ChatGoogleGenerativeAI(model="gemini-pro")
# repo_id = "mistralai/Mistral-7B-v0.1"
# model = HuggingFaceHub(huggingfacehub_api_token=os.environ["HUGGIGFACEHUB_API_KEY"], repo_id=repo_id, model_kwargs={"temperature":0.2, "max_new_tokens":50})
retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 6})

qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 6}),
    return_source_documents=False,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [15]:
def ask(question):
    print(qa_chain({"query": question})['result'])

In [19]:
while(True):
  question = input("Please ask a question: ")
  if question == '':
    break
  else:
    clear_output(wait=True)
    ask(question)
    #time.sleep(1)